In [1]:
from IPython.core.display import display, HTML
display(HTML(
"""<style>
div.container { width:100% !important; }
div.CodeMirror {font-family: Consolas; font-size: 16pt;}
div.output { font-size: 16pt; font-weight: bold;}
div.input { font-family: Consolas; font-size: 16pt;}
div.prompt { min-width: 100px;}
</style>
"""))
import warnings
warnings.filterwarnings(action='ignore')

# DNN을 이용한 분류

In [2]:
import tensorflow as tf

In [3]:
import numpy as np
import pandas as pd

In [4]:
import cv2
img = cv2.imread('images/digits.png', cv2.IMREAD_GRAYSCALE)
print(img.shape)

(1000, 2000)


In [5]:
cells = [np.hsplit(row,100) for row in np.vsplit(img,50)]
x = np.array(cells) / 255.
X = x[:,:].reshape(-1,400).astype(np.float32)
y = pd.get_dummies(np.repeat(np.arange(10),500)).to_numpy()

In [6]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)

In [7]:
# 텐서플로우 2.x이상일 경우 사용하세요
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [21]:
x = tf.placeholder(tf.float32, [None, 400])
y = tf.placeholder(tf.float32, [None, 10])

In [34]:
W1 = tf.Variable(tf.truncated_normal([400, 512], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[512]))
h1 = tf.nn.relu(tf.matmul(x, W1)+b1)

In [35]:
W2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[256]))
h2 = tf.nn.relu(tf.matmul(h1, W2)+b2)

In [36]:
W = tf.Variable(tf.zeros([256, 10]))
b = tf.Variable(tf.zeros([10]))

In [37]:
h = tf.nn.softmax(tf.matmul(h2, W)+b)

In [38]:
cross_entropy = -tf.reduce_sum(y * tf.log(h), 
                               reduction_indices=[1])

In [39]:
loss = tf.reduce_mean(cross_entropy)

In [40]:
train = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [41]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [42]:
tot_cnt = len(train_X)

for epoch in range(1000):
    avg_loss = 0
    batch_size = 128
    tot_iter = int(tot_cnt / batch_size)
    for step in range(tot_iter):
        batch_X = train_X[int(step*batch_size):int((step+1)*batch_size)]
        batch_y = train_y[int(step*batch_size):int((step+1)*batch_size)]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if tot_cnt//tot_iter != 0:
        batch_X = train_X[int(tot_iter*batch_size):]
        batch_y = train_y[int(tot_iter*batch_size):]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_ / tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if epoch%100 == 0:
        print("step {}, train loss {}".format(epoch, avg_loss))

step 0, train loss 1.2996179351100214
step 100, train loss 0.0005773166307739706
step 200, train loss 0.00024096135947965436
step 300, train loss 0.00014706338468520831
step 400, train loss 0.00010413862091135579
step 500, train loss 7.985396533277474e-05
step 600, train loss 6.436073103773028e-05
step 700, train loss 5.367593446323924e-05
step 800, train loss 4.5889781244181076e-05
step 900, train loss 3.998081491671554e-05


In [ ]:
# for i in range(100):
#     _, loss_value = sess.run([train, loss], feed_dict={x: train_X, y: train_y})
#     if i % 10000 == 0:
#         print('Step %d, %.5f' % (i, loss_value))

In [43]:
correct_prediction = tf.equal(tf.argmax(h, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:test_X, y:test_y}))

0.9546667


In [44]:
print(sess.run(h, feed_dict={x:test_X})[0:5])

[[4.7012138e-14 1.0577590e-13 2.0417158e-14 2.9307328e-06 3.9842035e-10
  6.8072847e-10 4.7557567e-15 5.5278856e-06 3.9758341e-08 9.9999154e-01]
 [1.1149249e-16 1.7046755e-15 8.8527748e-11 3.8099952e-14 7.1449932e-22
  6.6125692e-17 7.7724398e-20 1.0000000e+00 1.0836062e-17 4.0714668e-11]
 [9.9934417e-01 2.5314607e-12 6.2491652e-04 1.8076485e-13 2.7769923e-18
  2.8503950e-06 1.8411626e-08 4.6750941e-12 2.7936605e-05 6.3573402e-10]
 [4.4414078e-10 7.3182511e-01 1.4946452e-06 3.7714673e-04 7.5664175e-08
  3.7142318e-08 3.4096692e-09 2.6497722e-01 1.9887026e-05 2.7990961e-03]
 [2.1112703e-09 4.1649251e-10 2.8274857e-07 9.9988830e-01 1.2248153e-19
  1.1147763e-04 1.1703716e-14 1.6792355e-14 5.6438587e-09 1.1240788e-10]]


In [45]:
predict_y = sess.run(tf.argmax(h, 1), feed_dict={x:test_X})
print(predict_y[0:20])

[9 7 0 1 3 8 7 8 9 7 2 6 9 7 0 3 9 3 7 3]


In [46]:
print(test_y.argmax(axis=1)[0:20])

[9 7 0 9 3 8 7 8 9 7 2 6 9 7 0 3 9 3 7 5]


In [47]:
cross_tab = pd.crosstab(test_y.argmax(axis=1)+3, predict_y+3, margins=True)
cross_tab

col_0,3,4,5,6,7,8,9,10,11,12,All
row_0,,,,,,,,,,,
3,173,0,0,0,0,0,3,1,0,2,179
4,0,134,0,0,0,1,0,1,0,1,137
5,0,3,146,1,1,0,1,0,1,0,153
6,0,0,5,138,0,3,0,1,1,1,149
7,0,0,3,0,140,0,0,0,1,3,147
8,0,1,1,2,1,140,0,1,1,0,147
9,1,0,0,0,0,3,133,0,0,0,137
10,0,1,0,0,0,0,0,141,0,6,148
11,0,1,2,3,1,0,1,0,136,1,145


# 케라스를 이용한 분류

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import cv2
img = cv2.imread('images/digits.png', cv2.IMREAD_GRAYSCALE)   
print(img.shape)

In [ ]:
cells = [np.hsplit(row,100) for row in np.vsplit(img,50)]
x = np.array(cells)
X = x[:,:].reshape(-1,400).astype(np.float32)
y = pd.get_dummies(np.repeat(np.arange(10),500)).to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)

In [ ]:
train_X.shape, train_y.shape

In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [ ]:
from tensorflow.keras.layers import Input, Dense
model.add(Input(400))
model.add(Dense(512, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(10, activation="softmax"))

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", 
              metrics=["accuracy"])

In [ ]:
model.fit(train_X, train_y, batch_size=200, epochs=50, verbose=1)

In [ ]:
pred = np.argmax(model.predict(test_X), axis=1)
pred[:10]

In [ ]:
np.argmax(test_y, axis=1)[:10] # 테스트 데이터의 정답

In [ ]:
pd.crosstab(np.argmax(test_y, axis=1), pred)

In [ ]:
model.evaluate(test_X, test_y)